In [1]:
import pandas as pd, numpy as np
from pathlib import Path

# ---------- paths ----------
CWD = Path.cwd()
DATA = (CWD / "../data") if (CWD / "../data").exists() else (CWD / "data")

def read_any(name):
    """Read CSV by common casings; raise if not found."""
    for base in [name, name.capitalize(), name.lower()]:
        p = DATA / f"{base}.csv"
        if p.exists():
            print("reading:", p.name)
            return pd.read_csv(p, low_memory=False)
    raise FileNotFoundError(f"{name}.csv not found in {DATA}")

def norm_cols(df):
    df = df.copy()
    df.columns = (df.columns
        .str.strip().str.lower()
        .str.replace(r"[^a-z0-9]+", "_", regex=True)
        .str.strip("_")
    )
    return df

def pick(df, options):
    for c in options:
        if c in df.columns: return c
    return None

# ---------- read ----------
items = norm_cols(read_any("items"))
sales = norm_cols(read_any("Sales"))
stores = norm_cols(read_any("Supermarkets"))
promo  = norm_cols(read_any("Promotion"))

# ---------- keys ----------
code_s  = pick(sales,  ["code","item_code","product_code"])
code_i  = pick(items,  ["code","item_code","product_code"])
store_s = pick(sales,  ["supermarket_no","store_no","store","store_id"])
store_t = pick(stores, ["supermarket_no","store_no","store","store_id"])

if not code_s or not code_i:
    raise ValueError("Could not find item code in sales/items.")

# ---------- amount ----------
amt_s = pick(sales, ["amount","sales_amount","value","revenue"])
if not amt_s:
    qty = pick(sales, ["quantity","qty","units"])
    price = pick(sales, ["price","unit_price"])
    if qty and price:
        sales["amount"] = pd.to_numeric(sales[qty], errors="coerce") * \
                          pd.to_numeric(sales[price], errors="coerce")
    else:
        raise ValueError("No 'amount' and cannot compute (need quantity × price).")
else:
    sales["amount"] = pd.to_numeric(sales[amt_s], errors="coerce")

# ---------- datetime on sales ----------
date_s = pick(sales, ["transaction_time","time_of_transactions","transaction_date",
                      "date","datetime","timestamp","time"])
if date_s:
    sdt = pd.to_datetime(sales[date_s], errors="coerce", infer_datetime_format=True)
    sales["transaction_time"] = sdt
    sales["year_week"] = sdt.dt.strftime("%G-%V")

# ---------- dedupe basics ----------
for df in [items, sales, stores, promo]:
    df.drop_duplicates(inplace=True)

# ---------- joins ----------
master = sales.merge(items.rename(columns={code_i:"code"}), left_on=code_s, right_on="code", how="left")

if store_s and store_t:
    master = master.merge(
        stores.rename(columns={store_t:"supermarket_no"}),
        left_on=store_s, right_on="supermarket_no", how="left"
    )

# add promotions if we can align by item/store/week
if {"year_week","code","supermarket_no"}.issubset(master.columns):
    # try standardizing promo keys
    code_p  = pick(promo,  ["code","item_code","product_code"])
    store_p = pick(promo,  ["supermarket_no","store_no","store","store_id"])
    if code_p and store_p:
        # attempt to build promo week
        week_p = pick(promo, ["year_week","week"])
        if week_p != "year_week" and "week" in promo.columns:
            promo["week"] = pd.to_numeric(promo["week"], errors="coerce")
        if "year_week" not in promo.columns and {"year","week"}.issubset(promo.columns):
            y = pd.to_numeric(promo["year"], errors="coerce").astype("Int64").astype(str)
            w = pd.to_numeric(promo["week"], errors="coerce").astype("Int64").astype(str).str.zfill(2)
            promo["year_week"] = (y + "-" + w)
        if "year_week" in promo.columns:
            jcols = ["year_week","code","supermarket_no"]
            promo_std = promo.rename(columns={code_p:"code", store_p:"supermarket_no"})
            promo_std = promo_std[jcols + [c for c in promo_std.columns if c not in jcols]]
            master = master.merge(promo_std, on=jcols, how="left")

# ---------- quick feature flags ----------
for c in ["feature","display"]:
    if c in master.columns:
        master[c] = master[c].astype(str).str.strip().replace({"nan":"0","None":"0"})
master["promo_flag"] = 0
for c in ["feature","display"]:
    if c in master.columns:
        master.loc[master[c].ne("0"), "promo_flag"] = 1

# ---------- save outputs (ignored by git) ----------
( DATA / "cleaned_items.csv").write_text(items.to_csv(index=False),  encoding="utf-8")
( DATA / "cleaned_sales.csv").write_text(sales.to_csv(index=False),  encoding="utf-8")
( DATA / "cleaned_supermarkets.csv").write_text(stores.to_csv(index=False), encoding="utf-8")
( DATA / "cleaned_promotion.csv").write_text(promo.to_csv(index=False), encoding="utf-8")
( DATA / "cleaned_master.csv").write_text(master.to_csv(index=False), encoding="utf-8")

# small sample for reviewers
master.head(50_000).to_csv(DATA / "cleaned_master_sample.csv", index=False)

print("Done. Shapes:",
      "items", items.shape, "| sales", sales.shape, "| stores", stores.shape,
      "| master", master.shape)


reading: items.csv
reading: Sales.csv
reading: Supermarkets.csv
reading: Promotion.csv


C:\Users\sik2k\AppData\Local\Temp\ipykernel_2416\4000941854.py:63: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  sdt = pd.to_datetime(sales[date_s], errors="coerce", infer_datetime_format=True)


Done. Shapes: items (927, 5) | sales (1048575, 13) | stores (387, 2) | master (1048575, 18)
